In [3]:
# Test all required packages
import neo4j
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
print("✅ All packages imported successfully")

# Test Neo4j connectivity  
from neo4j import GraphDatabase
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
with driver.session() as session:
    result = session.run("RETURN 'Environment Ready!' as message")
    print(f"✅ {result.single()['message']}")
driver.close()

✅ All packages imported successfully
✅ Environment Ready!


In [11]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import networkx as nx
import numpy as np
# Connect to Docker Neo4j instance
try:
    driver = GraphDatabase.driver("bolt://localhost:7687", 
                                 auth=("neo4j", "password"), 
                                 database="social")
    driver.verify_connectivity()
    print("✅ Connected to Neo4j Docker container successfully!")
    
    # Test query
    with driver.session() as session:
        result = session.run("RETURN 'Docker Neo4j connection works!' AS message")
        print(result.single()["message"])
        
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("Check Docker Neo4j container is running")

# Utility function for running queries and returning DataFrames
def run_query(query, database="social"):
    """Execute Cypher query and return results as pandas DataFrame"""
    with driver.session(database=database) as session:
        result = session.run(query)
        data = [record.data() for record in result]
        return pd.DataFrame(data)

# Test the function
test_df = run_query("MATCH (u:User) RETURN count(u) AS user_count")
print(f"✅ Data export function working. User count: {test_df.iloc[0]['user_count']}")

✅ Connected to Neo4j Docker container successfully!
Docker Neo4j connection works!
✅ Data export function working. User count: 5


In [13]:
# Export comprehensive user data
users_query = """
MATCH (u:User)
OPTIONAL MATCH (u)-[:POSTED]->(p:Post)
OPTIONAL MATCH (p)<-[:LIKES]-(liker:User)
OPTIONAL MATCH (follower:User)-[:FOLLOWS]->(u)
OPTIONAL MATCH (u)-[:INTERESTED_IN]->(topic:Topic)
WITH u, 
     count(DISTINCT p) AS posts_count,
     count(DISTINCT liker) AS total_likes_received,
     count(DISTINCT follower) AS followers,
     count(DISTINCT topic) AS interests_count
RETURN u.username AS username,
       u.age AS age,
       u.location AS location,
       followers,
       posts_count,
       total_likes_received,
       interests_count
ORDER BY followers DESC
"""

users_df = run_query(users_query)
print("User Data Shape:", users_df.shape)
print("\nUser Data Preview:")
print(users_df.head())

User Data Shape: (5, 7)

User Data Preview:
        username  age           location  followers  posts_count  \
0    alice_codes   25  San Francisco, CA          3            2   
1  carol_creates   28         London, UK          3            2   
2    eve_fitness   32    Los Angeles, CA          2            0   
3    bob_travels   30       New York, NY          1            2   
4    frank_music   27         Austin, TX          1            0   

   total_likes_received  interests_count  
0                     2                3  
1                     2                2  
2                     0                1  
3                     1                3  
4                     0                2  


In [14]:
# Export relationship network
relationships_query = """
MATCH (u1:User)-[r:FOLLOWS]->(u2:User)
RETURN u1.username AS from_user,
       u2.username AS to_user,
       r.since AS relationship_since,
       'FOLLOWS' AS relationship_type
"""

relationships_df = run_query(relationships_query)
print("Relationships Data Shape:", relationships_df.shape)
print("\nRelationships Preview:")
print(relationships_df.head())

Relationships Data Shape: (10, 4)

Relationships Preview:
     from_user        to_user                   relationship_since  \
0  alice_codes    bob_travels  2020-06-15T14:30:00.000000000+00:00   
1  alice_codes  carol_creates  2021-01-20T16:45:00.000000000+00:00   
2  bob_travels    alice_codes  2020-06-16T10:00:00.000000000+00:00   
3  bob_travels  carol_creates  2021-03-05T09:15:00.000000000+00:00   
4  bob_travels    eve_fitness  2021-05-12T14:30:00.000000000+00:00   

  relationship_type  
0           FOLLOWS  
1           FOLLOWS  
2           FOLLOWS  
3           FOLLOWS  
4           FOLLOWS  


In [16]:
G = nx.from_pandas_edgelist(relationships_df, 
                           source='from_user', 
                           target='to_user',
                           create_using=nx.DiGraph())

# Add node attributes from user data
for _, row in users_df.iterrows():
    if row['username'] in G.nodes():
        G.nodes[row['username']]['location'] = row['location']
        G.nodes[row['username']]['followers'] = row['followers']
        G.nodes[row['username']]['total_likes'] = row['total_likes_received']

# Generate optimal layout for visualization
pos = nx.spring_layout(G, k=3, iterations=50)

print(f"✅ Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
print(f"✅ Layout generated for {len(pos)} positions")

✅ Graph created with 5 nodes and 10 edges
✅ Layout generated for 5 positions


In [20]:
# Create edge traces for network connections
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(x=edge_x, y=edge_y,
                       line=dict(width=2, color='lightgray'),
                       hoverinfo='none',
                       mode='lines')

# Create node traces with interactive features
node_x = []
node_y = []
node_text = []
node_color = []
node_size = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    # Get node attributes
    followers = G.nodes[node].get('followers', 0)
    location = G.nodes[node].get('location', 'Unknown')
    total_likes = G.nodes[node].get('total_likes', 0)
    
    node_text.append(f"{node}<br>Followers: {followers}<br>Location: {location}<br>Likes: {total_likes}")
    node_color.append(followers)
    node_size.append(max(15, followers * 2))  # Scale node size by followers

node_trace = go.Scatter(x=node_x, y=node_y,
                       mode='markers+text',
                       hoverinfo='text',
                       text=[node for node in G.nodes()],
                       hovertext=node_text,
                       textposition="middle center",
                       marker=dict(size=node_size,
                                 color=node_color,
                                 colorscale='blues',
                                 showscale=True,
                                 colorbar=dict(title="Followers")))

# Create and display the interactive figure
fig = go.Figure(data=[edge_trace, node_trace],
               layout=go.Layout(
                title=dict(text='Interactive Social Network Graph', font=dict(size=16)),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Node size = followers, color = followers count",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002,
                    xanchor="left", yanchor="bottom",
                    font=dict(color="gray", size=12)
                )],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.show()

# Alternative display methods if above doesn't work
print("✅ Network graph created successfully!")
print(f"Graph contains {len(node_x)} nodes and {len(edge_x)//3} edges")

# If graph doesn't display, try these alternatives:
fig.show(renderer="browser")  # Opens in browser
# fig.write_html("network_graph.html")  # Saves to file

✅ Network graph created successfully!
Graph contains 5 nodes and 10 edges


In [21]:
# Create geographic distribution chart
location_counts = users_df['location'].value_counts()

fig_geo = px.bar(x=location_counts.index, 
                 y=location_counts.values,
                 title='User Distribution by Location',
                 labels={'x': 'Location', 'y': 'Number of Users'},
                 color=location_counts.values,
                 color_continuous_scale='blues')

fig_geo.update_layout(showlegend=False, 
                     xaxis_title="Location",
                     yaxis_title="Number of Users")
fig_geo.show()

# Geographic influence analysis
geo_influence = users_df.groupby('location').agg({
    'followers': 'mean',
    'total_likes_received': 'mean',
    'interests_count': 'mean'
}).round(2)

print("\nInfluence by Location:")
print(geo_influence)


Influence by Location:
                   followers  total_likes_received  interests_count
location                                                           
Austin, TX               1.0                   0.0              2.0
London, UK               3.0                   2.0              2.0
Los Angeles, CA          2.0                   0.0              1.0
New York, NY             1.0                   1.0              3.0
San Francisco, CA        3.0                   2.0              3.0


In [24]:
# Create comprehensive dashboard
fig_dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=('User Age Distribution', 'Followers vs Engagement', 
                   'Interest Diversity', 'Network Metrics Summary'),
    specs=[[{"type": "histogram"}, {"type": "scatter"}],
           [{"type": "bar"}, {"type": "table"}]]
)

# Age distribution
fig_dashboard.add_trace(
    go.Histogram(x=users_df['age'], name='Age Distribution', nbinsx=10),
    row=1, col=1
)

# Followers vs engagement scatter
fig_dashboard.add_trace(
    go.Scatter(x=users_df['followers'], 
              y=users_df['total_likes_received'],
              mode='markers+text',
              text=users_df['username'],
              textposition="top center",
              name='Engagement Analysis',
              marker=dict(size=12, color='blue')),
    row=1, col=2
)

# Interest diversity by location
location_interests = users_df.groupby('location')['interests_count'].mean()
fig_dashboard.add_trace(
    go.Bar(x=location_interests.index, y=location_interests.values, name='Avg Interests'),
    row=2, col=1
)

# Network summary table
summary_data = [
    ['Total Users', len(users_df)],
    ['Total Connections', len(relationships_df)],
    ['Average Followers', round(users_df['followers'].mean(), 1)],
    ['Most Active Location', users_df['location'].value_counts().index[0]],
    ['Network Density', round(len(relationships_df) / (len(users_df) * (len(users_df) - 1)), 3)]
]

fig_dashboard.add_trace(
    go.Table(
        header=dict(values=['Metric', 'Value']),
        cells=dict(values=list(zip(*summary_data)))
    ),
    row=2, col=2
)

fig_dashboard.update_layout(height=800, showlegend=False, 
                           title_text="Social Network Analytics Dashboard")
fig_dashboard.show()

# Alternative display methods if dashboard doesn't appear
print("✅ Dashboard created successfully!")
print(f"Dashboard contains {len(users_df)} users across {len(location_interests)} locations")

# If dashboard doesn't display, try these alternatives:
fig_dashboard.show(renderer="browser")  # Opens in browser
# fig_dashboard.write_html("analytics_dashboard.html")  # Saves to file

# Debug: Check if data exists for charts
print(f"Age data available: {not users_df['age'].empty}")
print(f"Location data available: {len(location_interests) > 0}")
print(f"Summary data rows: {len(summary_data)}")

✅ Dashboard created successfully!
Dashboard contains 5 users across 5 locations
Age data available: True
Location data available: True
Summary data rows: 5


In [25]:
# Advanced network analysis
def calculate_network_metrics():
    """Calculate comprehensive network metrics"""
    
    # Basic metrics
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    density = nx.density(G)
    
    # Degree analysis
    degrees = dict(G.degree())
    avg_degree = sum(degrees.values()) / len(degrees)
    
    # Centrality measures
    betweenness = nx.betweenness_centrality(G)
    
    # Find most influential users
    followers_dict = {user: data['followers'] for user, data in G.nodes(data=True)}
    following_dict = dict(G.out_degree())
    
    most_followers = max(followers_dict.items(), key=lambda x: x[1])
    most_following = max(following_dict.items(), key=lambda x: x[1])
    most_between = max(betweenness.items(), key=lambda x: x[1])
    
    return {
        'nodes': num_nodes,
        'edges': num_edges,
        'density': round(density, 3),
        'avg_degree': round(avg_degree, 2),
        'most_followed_user': most_followers[0],
        'most_active_user': most_following[0],
        'most_bridge_user': most_between[0]
    }

network_metrics = calculate_network_metrics()

print("=== NETWORK ANALYSIS RESULTS ===")
for key, value in network_metrics.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

=== NETWORK ANALYSIS RESULTS ===
Nodes: 5
Edges: 10
Density: 0.5
Avg Degree: 4.0
Most Followed User: alice_codes
Most Active User: bob_travels
Most Bridge User: alice_codes


In [26]:
# Analyze content performance
content_query = """
MATCH (p:Post)
OPTIONAL MATCH (p)<-[:LIKES]-(liker:User)
OPTIONAL MATCH (p)<-[:POSTED]-(author:User)
OPTIONAL MATCH (p)-[:TAGGED_WITH]->(topic:Topic)
RETURN p.content AS content,
       author.username AS author,
       p.likes AS like_count,
       count(DISTINCT liker) AS actual_likes,
       count(DISTINCT topic) AS topic_count,
       p.timestamp AS post_time
ORDER BY p.likes DESC
"""

content_df = run_query(content_query)

print("=== CONTENT PERFORMANCE ANALYSIS ===")
print(f"Total Posts: {len(content_df)}")
print(f"Average Likes per Post: {content_df['like_count'].mean():.1f}")
print(f"Most Liked Post: {content_df.iloc[0]['like_count']} likes")
print(f"Top Performing Author: {content_df.iloc[0]['author']}")

# Create content performance visualization
fig_content = px.bar(content_df.head(10), 
                    x='author', 
                    y='like_count',
                    title='Top 10 Posts by Engagement',
                    labels={'like_count': 'Number of Likes', 'author': 'Post Author'},
                    color='like_count',
                    color_continuous_scale='viridis')

fig_content.update_layout(xaxis_tickangle=-45)
fig_content.show()

=== CONTENT PERFORMANCE ANALYSIS ===
Total Posts: 6
Average Likes per Post: 43.7
Most Liked Post: 67 likes
Top Performing Author: bob_travels


In [27]:
# Create comprehensive analysis summary
summary_stats = {
    'Total Users': len(users_df),
    'Total Relationships': len(relationships_df),
    'Average Followers': round(users_df['followers'].mean()),
    'Average Engagement': round(users_df['total_likes_received'].mean()),
    'Most Engaged User': users_df.loc[users_df['total_likes_received'].idxmax(), 'username'],
    'Most Connected City': users_df['location'].value_counts().index[0],
    'Network Density': round(len(relationships_df) / (len(users_df) * (len(users_df) - 1)), 3)
}

print("=== SOCIAL NETWORK ANALYSIS SUMMARY ===")
for key, value in summary_stats.items():
    print(f"{key}: {value}")

# Close Neo4j connection
driver.close()

=== SOCIAL NETWORK ANALYSIS SUMMARY ===
Total Users: 5
Total Relationships: 10
Average Followers: 2
Average Engagement: 1
Most Engaged User: alice_codes
Most Connected City: San Francisco, CA
Network Density: 0.5


In [28]:
import pandas as pd

tool_comparison = pd.DataFrame({
    'Tool': ['Neo4j Browser', 'Python/Jupyter'],
    'Best For': [
        'Technical query development and advanced visualization',
        'Advanced analytics and custom visualizations'
    ],
    'User Type': [
        'Developers, Data Engineers, Business Analysts',
        'Data Scientists, Analysts'
    ],
    'Strengths': [
        'Cypher development, Schema exploration, Interactive visualization',
        'Statistical analysis, Custom dashboards'
    ],
    'Use Cases': [
        'Query optimization, Data modeling, Business presentations',
        'Machine learning, Complex analytics'
    ]
})

print("\n=== DATA VISUALIZATION WORKFLOW GUIDE ===")
print(tool_comparison.to_string(index=False))


=== DATA VISUALIZATION WORKFLOW GUIDE ===
          Tool                                               Best For                                     User Type                                                         Strengths                                                 Use Cases
 Neo4j Browser Technical query development and advanced visualization Developers, Data Engineers, Business Analysts Cypher development, Schema exploration, Interactive visualization Query optimization, Data modeling, Business presentations
Python/Jupyter           Advanced analytics and custom visualizations                     Data Scientists, Analysts                           Statistical analysis, Custom dashboards                       Machine learning, Complex analytics
